In [1]:
import dgl
import torch
import numpy as np
import matplotlib
import math
from matplotlib import pyplot as plt
from matplotlib.tri import Triangulation
from IPython.display import display, clear_output
import os
import wandb
import copy

savelen_global = 6

Using backend: pytorch


In [2]:
random_seed = 19960926
np.random.seed(random_seed)

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

dgl.seed(random_seed)
dgl.random.seed(random_seed)

In [3]:
def collate(samples):
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    num_nodes = batched_graph.batch_num_nodes()
    return batched_graph, torch.tensor(np.vstack(labels))

def count_param(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def evaluate(model, loader):
    model.eval()
    with torch.no_grad():
        for disjoint_graphs_test, output_features_test in loader:
            test_features_unchange = disjoint_graphs_test.ndata['Features'][:,0:5]
            test_features_input = disjoint_graphs_test.ndata['Features'][:,5].reshape(disjoint_graphs_test.ndata['Features'].shape[0],1)
            graphs4test = copy.copy(disjoint_graphs_test)
            test_label = copy.deepcopy(output_features_test[:,savelen_global-2]).reshape(disjoint_graphs_test.ndata['Features'].shape[0],1)
            for seq_batch in range(savelen_global-1):
                if seq_batch==0:
                    test_features = copy.deepcopy(test_features_input)
                else:
                    test_features = copy.deepcopy(pred_features.to('cpu').detach().numpy())
                graphs4test.ndata['Features'] = torch.tensor(np.concatenate((test_features_unchange,test_features),axis=1))
                pred_features = model(graphs4test.to('cuda'), graphs4test.ndata['Features'].to('cuda'))
                
                if seq_batch==savelen_global-2:
                    loss_test = MAE_loss(pred_features, test_label.to('cuda'))
            break
    return loss_test.to('cpu').detach().numpy()

def showMeshPlot_seq(nodes,elements,inputs,truth,pred):
    x = nodes[:,0]
    y = nodes[:,1]
    
    fig = plt.figure(figsize=(20,20))
    
    ax = fig.add_subplot(1,3,1)
    ax.set_aspect('equal')
    xy = np.c_[x,y]
    verts= xy[elements]
    pc = matplotlib.collections.PolyCollection(verts,edgecolor='black',cmap='Greys')
    pc.set_array(inputs)
    ax.add_collection(pc)
    ax.autoscale()
    
    ax = fig.add_subplot(1,3,2)
    ax.set_aspect('equal')
    xy = np.c_[x,y]
    verts= xy[elements]
    pc = matplotlib.collections.PolyCollection(verts,edgecolor='black',cmap='Greys')
    pc.set_array(truth)
    ax.add_collection(pc)
    ax.autoscale()
    
    ax = fig.add_subplot(1,3,3)
    ax.set_aspect('equal')
    xy = np.c_[x,y]
    verts= xy[elements]
    pc = matplotlib.collections.PolyCollection(verts,edgecolor='black',cmap='Greys')
    pc.set_array(pred)
    ax.add_collection(pc)
    ax.autoscale()

def plot_save(model,epoch,savefoldername):
    path = '/root/GNN/0_Own/data'
    foldermat = ['cantilever','curved','hook_mod','prlgram','winding','bending','trapezoid']
    os.makedirs('/root/GNN/0_Own/results' + '/' + savefoldername + '/' + '%d_epoch'%(epoch), exist_ok=True)
    
    for folder_name in foldermat:
        num_data = 500
        with open(path  + '/' + folder_name + '_folders' + '/' + folder_name + '/' + 'test_index.bin','rb') as test_ind:
            plot_test_ind = np.fromfile(test_ind,dtype = 'float32',count=-1).astype(int)
        
        with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '/' + 'ptadj.bin','rb') as infomat:
            length_p_t_adj = np.fromfile(infomat,dtype = 'float32',count=-1).reshape(3,num_data).transpose([1,0]).astype(int)

        randind = np.random.choice(plot_test_ind, replace=False)
        coor_num = length_p_t_adj[randind-1,0]
        node_num = length_p_t_adj[randind-1,1]
        adj_num = length_p_t_adj[randind-1,2]
        save_rho_length = savelen_global

        with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '/' + 'p_%d.bin'%(randind),'rb') as ptest:
            p = np.fromfile(ptest,dtype='float32',count=-1).reshape(2,coor_num).transpose([1,0])
       
        with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '/' + 't_%d.bin'%(randind),'rb') as ttest:
            t = np.fromfile(ttest,dtype='float32',count=-1).reshape(3,node_num).transpose([1,0]).astype(int)

        with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '/' + 'adj_%d.bin'%(randind),'rb') as adj:
            adj_list = np.fromfile(adj,dtype='float32',count=-1).reshape(2,adj_num).transpose([1,0])
            adj_list = adj_list.astype(int)
            adj_list += -1
            
        graph_plot = dgl.graph((adj_list[:,0],adj_list[:,1]))
        graph_plot = dgl.add_self_loop(graph_plot)
        graph_plot = dgl.to_bidirected(graph_plot,copy_ndata = True)
        
        if folder_name in ['cantilever','curved','hook','prlgram','winding']:
            with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '_seq' + '/' + folder_name + '_u_%d.bin'%(randind),'rb') as feat:
                features = np.fromfile(feat,dtype='float32',count=-1).reshape(2,node_num).transpose([1,0])

            with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '_stress' + '/' + 'stress_%d.bin'%(randind),'rb') as sig:
                stress = np.fromfile(sig,dtype='float32',count=-1).reshape(3,node_num).transpose([1,0])

            with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '_%dseq'%(savelen_global) + '/' + folder_name + '_%d.bin'%(randind)) as rhos:
                rho_mat = np.fromfile(rhos,dtype='float32',count=-1).reshape(save_rho_length,node_num).transpose([1,0])
                rho_mat[:,0] = np.mean(rho_mat[:,1])
                
            features = np.concatenate((features,stress),axis=1)

        elif folder_name in ['bending','trapezoid','hook_mod']:
            with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '_seq' + '/' + folder_name + '_features_%d.bin'%(randind),'rb') as feat:
                features = np.fromfile(feat,dtype='float32',count=-1).reshape(5,node_num).transpose([1,0])

            with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '_%dseq'%(savelen_global) + '/' + folder_name + '_%d.bin'%(randind)) as rhos:
                rho_mat = np.fromfile(rhos,dtype='float32',count=-1).reshape(save_rho_length,node_num).transpose([1,0])


        for rho_iter in range(savelen_global-1):
            if rho_iter==0:
                rho_inputs = rho_mat[:,0].reshape(node_num,1)
                node_label = rho_mat[:,1].reshape(node_num,1)
            else:
                rho_inputs = copy.deepcopy(predictions_plot.detach().numpy())
                node_label = rho_mat[:,rho_iter+1].reshape(node_num,1)

            node_features = np.concatenate((features,rho_inputs),axis=1)
            graph_plot.ndata['Features'] = torch.tensor(node_features)

            model = model.to('cpu')
            model.eval()
            with torch.no_grad():
                predictions_plot = model(graph_plot,graph_plot.ndata['Features'])

            showMeshPlot_seq(p,t-1,node_features[:,5].reshape(-1),node_label.reshape(-1),predictions_plot.reshape(-1))
            plt.savefig('/root/GNN/0_Own/results/' + savefoldername + '/' + '%d_epoch'%(epoch) + '/' + folder_name+'_seq_%d.jpg'%(rho_iter+1))
            plt.close()

    model = model.to('cuda')


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
def reg_layer(layer_num):
    assert len(np.array(layer_num).shape) == 1
    num_reg_layer = len(layer_num)
    namespace = []
    for ln in range(len(layer_num)):
        namespace.append('gat%d.fc'%(layer_num[ln])+'.weight')
    return namespace

In [4]:
class CosineAnnealingWarmUpRestarts(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2 
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [5]:
class Seq2Graph_train(dgl.data.DGLDataset):
    def __init__(self):
        super().__init__(name='MESH2GRAPH'+ '__' + 'train')

    def process(self):
        self.graphs = []
        self.labels = []

        path = '/root/GNN/0_Own/data'
        folder_list = ['cantilever','curved','hook_mod','prlgram','winding','bending','trapezoid']
        num_data = 500

        for folder_name in folder_list:
            indexmat = np.linspace(1,num_data,num_data).astype(int)
            
            with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '/' + 'ptadj.bin','rb') as infomat:
                length_p_t_adj = np.fromfile(infomat,dtype = 'float32',count=-1).reshape(3,num_data).transpose([1,0]).astype(int)

            with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '/' + 'test_index.bin','rb') as test_ind:
                part_test_ind = np.fromfile(test_ind,dtype = 'float32',count=-1).astype(int)
            
            part_train_ind = np.setdiff1d(indexmat, part_test_ind)
            numtrain = 0
            
            for data_iter in part_train_ind:
                numtrain = numtrain+1
                node_num = int(length_p_t_adj[data_iter-1,1])
                adj_num = int(length_p_t_adj[data_iter-1,2])
                save_rho_length = savelen_global
                
                with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '/' + 'adj_%d.bin'%(data_iter),'rb') as adj:
                    adj_list = np.fromfile(adj,dtype='float32',count=-1).reshape(2,adj_num).transpose([1,0])
                    adj_list = adj_list.astype(int)
                    adj_list += -1
                    
                if folder_name in ['cantilever','curved','hook','prlgram','winding']:
                    with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '_seq' + '/' + folder_name + '_u_%d.bin'%(data_iter),'rb') as feat:
                        features = np.fromfile(feat,dtype='float32',count=-1).reshape(2,node_num).transpose([1,0])

                    with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '_stress' + '/' + 'stress_%d.bin'%(data_iter),'rb') as sig:
                        stress = np.fromfile(sig,dtype='float32',count=-1).reshape(3,node_num).transpose([1,0])

                    with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '_%dseq'%(savelen_global) + '/' + folder_name + '_%d.bin'%(data_iter)) as rhos:
                        rho_mat = np.fromfile(rhos,dtype='float32',count=-1).reshape(save_rho_length,node_num).transpose([1,0])
                        rho_mat[:,0] = np.mean(rho_mat[:,1])
                        
                    node_features = np.concatenate((features,stress,rho_mat[:,0:savelen_global-1]),axis=1)
                    node_label = rho_mat[:,1:savelen_global]

                elif folder_name in ['bending','trapezoid','hook_mod']:
                    with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '_seq' + '/' + folder_name + '_features_%d.bin'%(data_iter),'rb') as feat:
                        features = np.fromfile(feat,dtype='float32',count=-1).reshape(5,node_num).transpose([1,0])

                    with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '_%dseq'%(savelen_global) + '/' + folder_name + '_%d.bin'%(data_iter)) as rhos:
                        rho_mat = np.fromfile(rhos,dtype='float32',count=-1).reshape(save_rho_length,node_num).transpose([1,0])
                        
                    node_features = np.concatenate((features,rho_mat[:,0:savelen_global-1]),axis=1)
                    node_label = rho_mat[:,1:savelen_global]

                assert rho_mat.shape[1]==savelen_global
                assert node_features.shape[1]==node_label.shape[1]+5

                g = dgl.graph((adj_list[:,0],adj_list[:,1]))
                g.ndata['Features'] = torch.tensor(node_features)
                g = dgl.add_self_loop(g)
                g = dgl.to_bidirected(g,copy_ndata = True)

                self.graphs.append(g)
                self.labels.append(node_label)
                                        
                print('Category Name : %s   Data Num : %d   Feature shape : (%d , %d)   Label shape : (%d , %d)'\
                      %(folder_name,numtrain,node_features.shape[0],node_features.shape[1],node_label.shape[0],node_label.shape[1]))
                clear_output(wait=True)
                    
        print('Graph Representation Done')
    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]
    
    def __len__(self):
        return len(self.graphs)

In [6]:
class Seq2Graph_test(dgl.data.DGLDataset):
    def __init__(self):
        super().__init__(name='MESH2GRAPH'+ '__' + 'test')

    def process(self):
        self.graphs = []
        self.labels = []

        path = '/root/GNN/0_Own/data'
        folder_list = ['cantilever','curved','hook_mod','prlgram','winding','bending','trapezoid']
        num_data = 500

        for folder_name in folder_list:
            with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '/' + 'ptadj.bin','rb') as infomat:
                length_p_t_adj = np.fromfile(infomat,dtype = 'float32',count=-1).reshape(3,num_data).transpose([1,0]).astype(int)

            with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '/' + 'test_index.bin','rb') as test_ind:
                part_test_ind = np.fromfile(test_ind,dtype = 'float32',count=-1).astype(int)
            
            numtest = 0
            for data_iter in part_test_ind:
                numtest = numtest+1
                node_num = int(length_p_t_adj[data_iter-1,1])
                adj_num = int(length_p_t_adj[data_iter-1,2])
                save_rho_length = savelen_global
                
                with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '/' + 'adj_%d.bin'%(data_iter),'rb') as adj:
                    adj_list = np.fromfile(adj,dtype='float32',count=-1).reshape(2,adj_num).transpose([1,0])
                    adj_list = adj_list.astype(int)
                    adj_list += -1
                    
                if folder_name in ['cantilever','curved','hook','prlgram','winding']:
                    with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '_seq' + '/' + folder_name + '_u_%d.bin'%(data_iter),'rb') as feat:
                        features = np.fromfile(feat,dtype='float32',count=-1).reshape(2,node_num).transpose([1,0])

                    with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '_stress' + '/' + 'stress_%d.bin'%(data_iter),'rb') as sig:
                        stress = np.fromfile(sig,dtype='float32',count=-1).reshape(3,node_num).transpose([1,0])

                    with open(path + '/' + folder_name + '_folders' + '/' + folder_name + '_%dseq'%(savelen_global) + '/' + folder_name + '_%d.bin'%(data_iter)) as rhos:
                        rho_mat = np.fromfile(rhos,dtype='float32',count=-1).reshape(save_rho_length,node_num).transpose([1,0])
                        rho_mat[:,0] = np.mean(rho_mat[:,1])

                    node_features = np.concatenate((features,stress,rho_mat[:,0:savelen_global-1]),axis=1)
                    node_label = rho_mat[:,1:savelen_global]
                    
                
                elif folder_name in ['bending','trapezoid','hook_mod']:
                    with open(path + '/' + folder_name + '_folders' + '/'  + folder_name + '_seq' + '/' + folder_name + '_features_%d.bin'%(data_iter),'rb') as feat:
                        features = np.fromfile(feat,dtype='float32',count=-1).reshape(5,node_num).transpose([1,0])

                    with open(path + '/' + folder_name + '_folders' + '/'  + folder_name + '_%dseq'%(savelen_global) + '/' + folder_name + '_%d.bin'%(data_iter)) as rhos:
                        rho_mat = np.fromfile(rhos,dtype='float32',count=-1).reshape(save_rho_length,node_num).transpose([1,0])
                    
                    node_features = np.concatenate((features,rho_mat[:,0:savelen_global-1]),axis=1)
                    node_label = rho_mat[:,1:savelen_global]

                assert rho_mat.shape[1]==savelen_global
                assert node_features.shape[1]==node_label.shape[1]+5

                g = dgl.graph((adj_list[:,0],adj_list[:,1]))
                g.ndata['Features'] = torch.tensor(node_features)
                g = dgl.add_self_loop(g)
                g = dgl.to_bidirected(g,copy_ndata = True)

                self.graphs.append(g)
                self.labels.append(node_label)
                
                print('Category Name : %s   Data Num : %d   Feature shape : (%d , %d)   Label shape : (%d , %d)'\
                      %(folder_name,numtest,node_features.shape[0],node_features.shape[1],node_label.shape[0],node_label.shape[1]))
                clear_output(wait=True)
                    
        print('Graph Representation Done')
    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]
    
    def __len__(self):
        return len(self.graphs)

In [7]:
class TOP_MODEL(torch.nn.Module):
    def __init__(self, num_layers, num_hidden_features):
        super().__init__()
        assert num_layers%2 ==0, 'Number of Layers is not Even : Skip connection operation is obscure'
        assert num_hidden_features < 512, 'Number of Hidden Features is too Large'
        
        self.num_layers = num_layers
        self.num_hidden_features = num_hidden_features
        self.att_head = 2

        for nl in range(1,self.num_layers+1):
            if nl==1:
                self.gat1 = dgl.nn.pytorch.conv.GATConv(in_feats=6,out_feats=self.num_hidden_features,num_heads=self.att_head,bias=False)
                self.bn1 = torch.nn.BatchNorm1d(num_features = self.num_hidden_features)
                self.act1 = torch.nn.ReLU()
            
            elif ((nl<=self.num_layers/2) and (nl%3==1)) or ((nl>self.num_layers/2) and (nl%3==0)):
                in_GCN_tmp = "self.gat%d"%(nl)
                in_BN_tmp = "self.bn%d"%(nl)
                in_ACT_tmp = "self.act%d"%(nl)
                
                GCN_tmp = "dgl.nn.pytorch.conv.GATConv(in_feats=self.num_hidden_features,out_feats=self.num_hidden_features,num_heads=self.att_head,bias=False)"
                BN_tmp = "torch.nn.BatchNorm1d(num_features = self.num_hidden_features)"
                ACT_tmp = "torch.nn.ReLU()"
                
                exec('%s = %s'%(in_GCN_tmp,GCN_tmp))
                exec('%s = %s'%(in_BN_tmp,BN_tmp))
                exec('%s = %s'%(in_ACT_tmp,ACT_tmp))
            
            elif ((nl<=self.num_layers/2) and (nl%3==2)) or ((nl>self.num_layers/2) and (nl%3==2)):
                in_GCN_tmp = "self.gat%d"%(nl)
                in_DO_tmp = "self.do%d"%(nl)
                in_ACT_tmp = "self.act%d"%(nl)

                GCN_tmp = "dgl.nn.pytorch.conv.GATConv(in_feats=self.num_hidden_features,out_feats=self.num_hidden_features,num_heads=self.att_head,bias=True)"
                DO_tmp = "torch.nn.Dropout(p=0.2)"
                ACT_tmp = "torch.nn.ReLU()"

                exec('%s = %s'%(in_GCN_tmp,GCN_tmp))
                exec('%s = %s'%(in_DO_tmp,DO_tmp))
                exec('%s = %s'%(in_ACT_tmp,ACT_tmp))

            elif ((nl<=self.num_layers/2) and (nl%3==0)) or ((nl>self.num_layers/2) and (nl%3==1)):
                in_GCN_tmp = "self.gat%d"%(nl)
                in_ACT_tmp = "self.act%d"%(nl)

                GCN_tmp = "dgl.nn.pytorch.conv.GATConv(in_feats=self.num_hidden_features,out_feats=self.num_hidden_features,num_heads=self.att_head,bias=True)"
                ACT_tmp = "torch.nn.ReLU()"

                exec('%s = %s'%(in_GCN_tmp,GCN_tmp))
                exec('%s = %s'%(in_ACT_tmp,ACT_tmp))

        self.gat_last = dgl.nn.pytorch.conv.GATConv(in_feats=self.num_hidden_features,out_feats=1,num_heads=self.att_head,bias=True)
        self.act_last = torch.nn.Sigmoid()
        
    def forward(self, graphs, features):
        for i in range(1,self.num_layers+1):
            if i==1:
                self.c1 = self.gat1(graphs, features).mean(dim=1)
                self.b1 = self.bn1(self.c1)
                self.a1 = self.act1(self.b1)
            elif (i<=self.num_layers/2) and (i%3==1):
                exec('self.c%d = self.gat%d(graphs, self.a%d).mean(dim=1)'%(i,i,i-1))
                exec('self.b%d = self.bn%d(self.c%d)'%(i,i,i))
                exec('self.a%d = self.act%d(self.b%d)'%(i,i,i))
            elif (i<=self.num_layers/2) and (i%3==2):
                exec('self.c%d = self.gat%d(graphs, self.a%d).mean(dim=1)'%(i,i,i-1))
                exec('self.d%d = self.do%d(self.c%d)'%(i,i,i))
                exec('self.a%d = self.act%d(self.d%d)'%(i,i,i))
            elif (i<=self.num_layers/2) and (i%3==0):
                exec('self.c%d = self.gat%d(graphs, self.a%d).mean(dim=1)'%(i,i,i-1))
                exec('self.a%d = self.act%d(self.c%d)'%(i,i,i))

            elif (i>self.num_layers/2) and (i%3==1):
                exec('self.c%d = self.gat%d(graphs, self.a%d).mean(dim=1)'%(i,i,i-1))
                exec('self.skip%d = self.a%d + self.c%d'%(i,self.num_layers+1-i,i))
                exec('self.a%d = self.act%d(self.skip%d)'%(i,i,i))
            elif (i>self.num_layers/2) and (i%3==2):
                exec('self.c%d = self.gat%d(graphs, self.a%d).mean(dim=1)'%(i,i,i-1))
                exec('self.d%d = self.do%d(self.c%d)'%(i,i,i))
                exec('self.skip%d = self.a%d + self.d%d'%(i,self.num_layers+1-i,i))
                exec('self.a%d = self.act%d(self.skip%d)'%(i,i,i))
            elif (i>self.num_layers/2) and (i%3==0):
                exec('self.c%d = self.gat%d(graphs, self.a%d).mean(dim=1)'%(i,i,i-1))
                exec('self.b%d = self.bn%d(self.c%d)'%(i,i,i))
                exec('self.skip%d = self.a%d + self.b%d'%(i,self.num_layers+1-i,i))
                exec('self.a%d = self.act%d(self.skip%d)'%(i,i,i))
            
        exec('self.c_last = self.gat_last(graphs, self.a%d).mean(dim=1)'%(self.num_layers))
        out = self.act_last(self.c_last)
        
        return out

In [8]:
train_dataset = Seq2Graph_train()

Graph Representation Done


In [9]:
test_dataset = Seq2Graph_test()

Graph Representation Done


In [10]:
print(train_dataset.__len__())
print(test_dataset.__len__())

2800
700


In [11]:
num_layers = 18
num_hidden_features = 128
model = TOP_MODEL(num_layers,num_hidden_features)
model = model.to('cuda')

In [12]:
train_loader = dgl.dataloading.pytorch.GraphDataLoader(train_dataset, collate_fn=collate, batch_size=128, shuffle=True)
test_loader = dgl.dataloading.pytorch.GraphDataLoader(test_dataset, collate_fn=collate, batch_size=128, shuffle=True)

In [13]:
optimizer = torch.optim.Adam(model.parameters(),lr=0)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=200, T_mult=1, eta_max=0.005, T_up=40, gamma=0.5)
MAE_loss = torch.nn.L1Loss()

In [14]:
reg_layer_num = []
for ln in range(1,num_layers+1):
    if (ln<=num_layers/2) and (ln%3==0):
        reg_layer_num.append(ln)
    elif (ln>num_layers/2) and (ln%3==1):
        reg_layer_num.append(ln)
reg_layer_namespace = reg_layer(reg_layer_num)
print(reg_layer_namespace)

['gat3.fc.weight', 'gat6.fc.weight', 'gat9.fc.weight', 'gat10.fc.weight', 'gat13.fc.weight', 'gat16.fc.weight']


In [15]:
emax = 500
min_test_err = [1]
modelsave_interval = 100
plotsave_interval = 100
l2reg_factor = 0.005
experiment_name = "V6_RNN_all_%dseq_hook_mod"%(savelen_global)
wandb.init(project="GNN_seq_FINAL",group=experiment_name)
config = wandb.config

save_fig_foldername = 'V6_RNN_all_%dseq_hook_mod'%(savelen_global)
save_interv_model_foldername = experiment_name + '_interv'
save_best_model_foldername = experiment_name + '_best'

intervmodel_path = '/root/GNN/0_Own/models/' + save_interv_model_foldername
bestmodel_path = '/root/GNN/0_Own/models/' + save_best_model_foldername
os.makedirs(intervmodel_path, exist_ok=True)
os.makedirs(bestmodel_path, exist_ok=True)

for e in range(1,emax+1):
    for batch_num, (disjoint_graphs, output_features) in enumerate(train_loader):
        train_features_unchange = copy.deepcopy(disjoint_graphs.ndata['Features'][:,0:5])
        train_features_change = copy.deepcopy(disjoint_graphs.ndata['Features'][:,5:])
        graphs4train = copy.deepcopy(disjoint_graphs)
        for seq_batch in range(savelen_global-1):
            if seq_batch==0:
                train_features = copy.deepcopy(train_features_change[:,seq_batch]).reshape(disjoint_graphs.ndata['Features'].shape[0],1)
            else:
                train_features = next_feat
            train_label = copy.deepcopy(output_features[:,seq_batch]).reshape(disjoint_graphs.ndata['Features'].shape[0],1)

            graphs4train.ndata['Features'] = torch.tensor(np.concatenate((train_features_unchange,train_features),axis=1))

            pred_features = model(graphs4train.to('cuda'), graphs4train.ndata['Features'].to('cuda'))
            loss_training = MAE_loss(pred_features, train_label.to('cuda'))
            loss_reg = 0
            for parameter in model.named_parameters():
                for i in range(len(reg_layer_namespace)):
                    if reg_layer_namespace[i] in parameter:
                        loss_reg += torch.sum(parameter[1]**2)
            loss_training = loss_training + l2reg_factor*loss_reg
            optimizer.zero_grad()
            loss_training.backward()
            optimizer.step()
            next_feat = copy.copy(pred_features).to('cpu').detach().numpy()
        
            print('Epoch : %d     Batch : %d     Seq : %d     Training loss : %f     reg loss : %f     Learning rate : %f'% \
                  (e,batch_num+1,seq_batch+1,loss_training,loss_reg,scheduler.get_lr()[0]))
            clear_output(wait = True)
            
    loss_test = evaluate(model,test_loader)
    scheduler.step()
    wandb.log({"training_loss":loss_training, "test_loss":loss_test, "learning_rate":scheduler.get_lr()[0]})
    
    if e%plotsave_interval ==0:
        plot_save(model,e,save_fig_foldername)
    if e%modelsave_interval == 0:
        torch.save(model.state_dict(),intervmodel_path + '/' + 'model_%d_epoch'%(e))
    if min_test_err > loss_test:
        torch.save(model.state_dict(),bestmodel_path + '/' + 'best_model')
        min_test_err = copy.copy([loss_test])

Epoch : 500     Batch : 22     Seq : 5     Training loss : 0.099155     reg loss : 0.000087     Learning rate : 0.000875
